# Harvard Dataverse Analysis Walkthrough

# Setup

## Select extract
In order for the template cells to query data from the correct repository, enter the repository name as `repository` and repository object type as `object_type`.

In [1]:
repository = 'dataverse'
object_type = 'datasets'

In [2]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

In [3]:
#see more rows and columns of output
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100) 

## Helper Functions

In [4]:
import os, sys
dir2 = os.path.abspath('../')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)

from utils import analysis
from utils.crosswalk import RepositoryExtract, property_crosswalk
from utils import accessors

# Summary Statistic Walkthroughs

Read in the repository .json file

In [5]:
df = pd.read_json(f'{repository}_{object_type}.json')

C:\Users\slabou\AppData\Roaming\Python\Python310\site-packages\pandas\core\tools\datetimes.py:530: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
C:\Users\slabou\AppData\Roaming\Python\Python310\site-packages\pandas\core\tools\datetimes.py:530: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)


In [6]:
df

,name,type,url,global_id,description,published_at,publisher,citationHtml,identifier_of_dataverse,name_of_dataverse,citation,storageIdentifier,keywords,subjects,fileCount,versionId,versionState,majorVersion,minorVersion,createdAt,updatedAt,contacts,publications,authors,geographicCoverage,page,producers,relatedMaterial,dataSources,contributor,date_of_collection,deposit_date,files,full_author_info,grant_info,kind_of_data,language,notes_text,num_downloads,publication_citation,terms_of_use,topic_class_value
0,Conformer models and training datasets,dataset,https://doi.org/10.7910/DVN/N4VLQL,doi:10.7910/DVN/N4VLQL,Here you can find pre-trained machine learning...,2021-02-11 22:02:13,Machine learning with conformer ensembles,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",ml_confs,Machine learning with conformer ensembles,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",s3://10.7910/DVN/N4VLQL,"[machine learning, computational drug discover...","[Chemistry, Computer and Information Science]",2496.0,231124.0,RELEASED,3.0,0.0,2021-01-22T17:11:55Z,2021-02-11T22:02:13Z,"[{'name': 'Axelrod, Simon', 'affiliation': 'Ha...","[{'citation': 'Axelrod, S. and Gomez-Bombarell...","[Axelrod, Simon, Gomez-Bombarelli, Rafael]",None,1.0,None,None,None,None,None,2021-01-22,"[{'name': 'all.tab', 'fileFormat': 'text/tab-s...","Axelrod, Simon (Harvard University) - ORCID: 0...",None,None,None,None,19.0,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",\n\n\n\n Waiver\...,None
1,GEOM,dataset,https://doi.org/10.7910/DVN/JNGTDF,doi:10.7910/DVN/JNGTDF,"Here you can find GEOM, a dataset with over 33...",2021-02-01 19:44:41,Machine learning with conformer ensembles,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",ml_confs,Machine learning with conformer ensembles,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",s3://10.7910/DVN/JNGTDF,"[computational chemistry, quantum chemistry, m...","[Chemistry, Computer and Information Science]",6.0,228748.0,RELEASED,2.0,0.0,2021-01-22T16:37:13Z,2021-02-10T19:47:58Z,"[{'name': 'Axelrod, Simon', 'affiliation': 'Ha...","[{'citation': 'Axelrod, S. and Gomez-Bombarell...","[Axelrod, Simon, Gomez-Bombarelli, Rafael]",None,1.0,None,None,None,None,None,2021-01-22,"[{'name': 'drugs_crude.msgpack.tar.gz', 'fileF...","Axelrod, Simon (Harvard University) - ORCID: 0...",None,None,None,None,2.0,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",\n\n\n\n Waiver\...,None
2,Replication Data for: Increasing Uptake of Soc...,dataset,https://doi.org/10.7910/DVN/RBZONT,doi:10.7910/DVN/RBZONT,Data and code for replication of the Harvard D...,2021-10-26 16:11:34,Data and Code for Increasing Uptake of Social ...,"Charles, Grace, 2021, ""Replication Data for: I...",social_distancing_surgo-ventures,Data and Code for Increasing Uptake of Social ...,"Charles, Grace, 2021, ""Replication Data for: I...",s3://10.7910/DVN/RBZONT,None,"[Medicine, Health and Life Sciences, Social Sc...",4.0,278265.0,RELEASED,1.0,1.0,2021-10-25T18:05:29Z,2021-10-27T16:39:00Z,"[{'name': 'Sgaier, Sema', 'affiliation': 'Surg...",[{'citation': 'HDSR: Increasing Uptake of Soci...,"[Charles, Grace]",None,1.0,None,None,None,None,None,2021-10-25,"[{'name': 'Codebook 4_3.tab', 'fileFormat': 't...","Charles, Grace (Surgo Ventures) - ORCID: 0000-...",None,None,None,None,0.0,"Charles, Grace, 2021, ""Replication Data for: I...",\n\n\n\n Waiver\...,None
3,Base data for publication - The association of...,dataset,https://doi.org/10.7910/DVN/BFXCAM,doi:10.7910/DVN/BFXCAM,Base data for publication - The association of...,2020-06-15 18:12:08,The association of Coronavirus Disease-19 mort...,"Nag, Swapnika; Puri, Ankur; Brooks, Nathan; Ga...",covid19bcgML,The association of Coronavirus Disease-19 mort...,"Nag, Swapnika; Puri, Ankur; Brooks, Nathan; Ga...",s3://10.7910/DVN/BFXCAM,None,"[Medicine, Health and Life Sciences]",1.0,197619.0,RELEASED,1.0,0.0,2020-06-13T07:24:16Z,2020-06-15T18:12:08Z,"[{'name': 'Nag, Swapnika', 'affiliation': ''},...",None,"[Nag, Swapnika, Puri, Ankur

### Artifacts of web scraping
It looks like we have file info (names and extensions) in 'files' column for datasets from Harvard dataverse. Example: https://doi.org/10.7910/DVN/N4VLQL
The ones that don't have file info seem to (based on a quick search) be harvested from other dataverse instances, such as this one: https://doi.org/10.5683/SP2/IKX9ML

Must be very slight differences in web page set up, meaning the web scraping didn't work for these datasets that were harvested into main dataverse. They're technically from *a* dataverse instance that got harvested into the main Harvard Dataverse portal, but they are not originally *Harvard* Dataverse deposits

In [7]:
files_assoc = df['files']
files_assoc

0      [{'name': 'all.tab', 'fileFormat': 'text/tab-s...
1      [{'name': 'drugs_crude.msgpack.tar.gz', 'fileF...
2      [{'name': 'Codebook 4_3.tab', 'fileFormat': 't...
3      [{'name': 'Consolidated data_vF.tab', 'fileFor...
4      [{'name': 'processed_data.zip', 'fileFormat': ...
                             ...                        
328                                                   []
329                                                   []
330                                                   []
331                                                   []
332    [{'name': 'Averaged MIR spectra of the manure ...
Name: files, Length: 333, dtype: object

In [8]:
#how many are not empty lists?
files_assoc_true = files_assoc[(files_assoc.str.len() == 0) == False]
files_assoc_true

0      [{'name': 'all.tab', 'fileFormat': 'text/tab-s...
1      [{'name': 'drugs_crude.msgpack.tar.gz', 'fileF...
2      [{'name': 'Codebook 4_3.tab', 'fileFormat': 't...
3      [{'name': 'Consolidated data_vF.tab', 'fileFor...
4      [{'name': 'processed_data.zip', 'fileFormat': ...
                             ...                        
306    [{'name': 'behav.zip', 'fileFormat': 'applicat...
312    [{'name': 'Holland_HousingSurvey_English.pdf',...
313    [{'name': '00_README.txt', 'fileFormat': 'text...
314    [{'name': '00_Read_Me.txt', 'fileFormat': 'tex...
332    [{'name': 'Averaged MIR spectra of the manure ...
Name: files, Length: 270, dtype: object

In [9]:
print(f'Of the {len(files_assoc)} items in the Dataverse Datasets extract, {len(files_assoc_true)} have associated files')

Of the 333 items in the Dataverse Datasets extract, 270 have associated files


For our purposes, we're interested only in the 270 records that have associated files, as these are most relevant for our analyses.

In [10]:
df_subset = df[(files_assoc.str.len() == 0) == False]
df_subset

,name,type,url,global_id,description,published_at,publisher,citationHtml,identifier_of_dataverse,name_of_dataverse,citation,storageIdentifier,keywords,subjects,fileCount,versionId,versionState,majorVersion,minorVersion,createdAt,updatedAt,contacts,publications,authors,geographicCoverage,page,producers,relatedMaterial,dataSources,contributor,date_of_collection,deposit_date,files,full_author_info,grant_info,kind_of_data,language,notes_text,num_downloads,publication_citation,terms_of_use,topic_class_value
0,Conformer models and training datasets,dataset,https://doi.org/10.7910/DVN/N4VLQL,doi:10.7910/DVN/N4VLQL,Here you can find pre-trained machine learning...,2021-02-11 22:02:13,Machine learning with conformer ensembles,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",ml_confs,Machine learning with conformer ensembles,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",s3://10.7910/DVN/N4VLQL,"[machine learning, computational drug discover...","[Chemistry, Computer and Information Science]",2496.0,231124.0,RELEASED,3.0,0.0,2021-01-22T17:11:55Z,2021-02-11T22:02:13Z,"[{'name': 'Axelrod, Simon', 'affiliation': 'Ha...","[{'citation': 'Axelrod, S. and Gomez-Bombarell...","[Axelrod, Simon, Gomez-Bombarelli, Rafael]",None,1.0,None,None,None,None,None,2021-01-22,"[{'name': 'all.tab', 'fileFormat': 'text/tab-s...","Axelrod, Simon (Harvard University) - ORCID: 0...",None,None,None,None,19.0,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",\n\n\n\n Waiver\...,None
1,GEOM,dataset,https://doi.org/10.7910/DVN/JNGTDF,doi:10.7910/DVN/JNGTDF,"Here you can find GEOM, a dataset with over 33...",2021-02-01 19:44:41,Machine learning with conformer ensembles,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",ml_confs,Machine learning with conformer ensembles,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",s3://10.7910/DVN/JNGTDF,"[computational chemistry, quantum chemistry, m...","[Chemistry, Computer and Information Science]",6.0,228748.0,RELEASED,2.0,0.0,2021-01-22T16:37:13Z,2021-02-10T19:47:58Z,"[{'name': 'Axelrod, Simon', 'affiliation': 'Ha...","[{'citation': 'Axelrod, S. and Gomez-Bombarell...","[Axelrod, Simon, Gomez-Bombarelli, Rafael]",None,1.0,None,None,None,None,None,2021-01-22,"[{'name': 'drugs_crude.msgpack.tar.gz', 'fileF...","Axelrod, Simon (Harvard University) - ORCID: 0...",None,None,None,None,2.0,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",\n\n\n\n Waiver\...,None
2,Replication Data for: Increasing Uptake of Soc...,dataset,https://doi.org/10.7910/DVN/RBZONT,doi:10.7910/DVN/RBZONT,Data and code for replication of the Harvard D...,2021-10-26 16:11:34,Data and Code for Increasing Uptake of Social ...,"Charles, Grace, 2021, ""Replication Data for: I...",social_distancing_surgo-ventures,Data and Code for Increasing Uptake of Social ...,"Charles, Grace, 2021, ""Replication Data for: I...",s3://10.7910/DVN/RBZONT,None,"[Medicine, Health and Life Sciences, Social Sc...",4.0,278265.0,RELEASED,1.0,1.0,2021-10-25T18:05:29Z,2021-10-27T16:39:00Z,"[{'name': 'Sgaier, Sema', 'affiliation': 'Surg...",[{'citation': 'HDSR: Increasing Uptake of Soci...,"[Charles, Grace]",None,1.0,None,None,None,None,None,2021-10-25,"[{'name': 'Codebook 4_3.tab', 'fileFormat': 't...","Charles, Grace (Surgo Ventures) - ORCID: 0000-...",None,None,None,None,0.0,"Charles, Grace, 2021, ""Replication Data for: I...",\n\n\n\n Waiver\...,None
3,Base data for publication - The association of...,dataset,https://doi.org/10.7910/DVN/BFXCAM,doi:10.7910/DVN/BFXCAM,Base data for publication - The association of...,2020-06-15 18:12:08,The association of Coronavirus Disease-19 mort...,"Nag, Swapnika; Puri, Ankur; Brooks, Nathan; Ga...",covid19bcgML,The association of Coronavirus Disease-19 mort...,"Nag, Swapnika; Puri, Ankur; Brooks, Nathan; Ga...",s3://10.7910/DVN/BFXCAM,None,"[Medicine, Health and Life Sciences]",1.0,197619.0,RELEASED,1.0,0.0,2020-06-13T07:24:16Z,2020-06-15T18:12:08Z,"[{'name': 'Nag, Swapnika', 'affiliation': ''},...",None,"[Nag, Swapnika, Puri, Ankur

In [11]:
#one row is has None for everything except files, which isn't useful
#delete this one row
df_use = df_subset[df_subset.url.notnull()]
df_use

,name,type,url,global_id,description,published_at,publisher,citationHtml,identifier_of_dataverse,name_of_dataverse,citation,storageIdentifier,keywords,subjects,fileCount,versionId,versionState,majorVersion,minorVersion,createdAt,updatedAt,contacts,publications,authors,geographicCoverage,page,producers,relatedMaterial,dataSources,contributor,date_of_collection,deposit_date,files,full_author_info,grant_info,kind_of_data,language,notes_text,num_downloads,publication_citation,terms_of_use,topic_class_value
0,Conformer models and training datasets,dataset,https://doi.org/10.7910/DVN/N4VLQL,doi:10.7910/DVN/N4VLQL,Here you can find pre-trained machine learning...,2021-02-11 22:02:13,Machine learning with conformer ensembles,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",ml_confs,Machine learning with conformer ensembles,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",s3://10.7910/DVN/N4VLQL,"[machine learning, computational drug discover...","[Chemistry, Computer and Information Science]",2496.0,231124.0,RELEASED,3.0,0.0,2021-01-22T17:11:55Z,2021-02-11T22:02:13Z,"[{'name': 'Axelrod, Simon', 'affiliation': 'Ha...","[{'citation': 'Axelrod, S. and Gomez-Bombarell...","[Axelrod, Simon, Gomez-Bombarelli, Rafael]",None,1.0,None,None,None,None,None,2021-01-22,"[{'name': 'all.tab', 'fileFormat': 'text/tab-s...","Axelrod, Simon (Harvard University) - ORCID: 0...",None,None,None,None,19.0,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",\n\n\n\n Waiver\...,None
1,GEOM,dataset,https://doi.org/10.7910/DVN/JNGTDF,doi:10.7910/DVN/JNGTDF,"Here you can find GEOM, a dataset with over 33...",2021-02-01 19:44:41,Machine learning with conformer ensembles,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",ml_confs,Machine learning with conformer ensembles,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",s3://10.7910/DVN/JNGTDF,"[computational chemistry, quantum chemistry, m...","[Chemistry, Computer and Information Science]",6.0,228748.0,RELEASED,2.0,0.0,2021-01-22T16:37:13Z,2021-02-10T19:47:58Z,"[{'name': 'Axelrod, Simon', 'affiliation': 'Ha...","[{'citation': 'Axelrod, S. and Gomez-Bombarell...","[Axelrod, Simon, Gomez-Bombarelli, Rafael]",None,1.0,None,None,None,None,None,2021-01-22,"[{'name': 'drugs_crude.msgpack.tar.gz', 'fileF...","Axelrod, Simon (Harvard University) - ORCID: 0...",None,None,None,None,2.0,"Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...",\n\n\n\n Waiver\...,None
2,Replication Data for: Increasing Uptake of Soc...,dataset,https://doi.org/10.7910/DVN/RBZONT,doi:10.7910/DVN/RBZONT,Data and code for replication of the Harvard D...,2021-10-26 16:11:34,Data and Code for Increasing Uptake of Social ...,"Charles, Grace, 2021, ""Replication Data for: I...",social_distancing_surgo-ventures,Data and Code for Increasing Uptake of Social ...,"Charles, Grace, 2021, ""Replication Data for: I...",s3://10.7910/DVN/RBZONT,None,"[Medicine, Health and Life Sciences, Social Sc...",4.0,278265.0,RELEASED,1.0,1.0,2021-10-25T18:05:29Z,2021-10-27T16:39:00Z,"[{'name': 'Sgaier, Sema', 'affiliation': 'Surg...",[{'citation': 'HDSR: Increasing Uptake of Soci...,"[Charles, Grace]",None,1.0,None,None,None,None,None,2021-10-25,"[{'name': 'Codebook 4_3.tab', 'fileFormat': 't...","Charles, Grace (Surgo Ventures) - ORCID: 0000-...",None,None,None,None,0.0,"Charles, Grace, 2021, ""Replication Data for: I...",\n\n\n\n Waiver\...,None
3,Base data for publication - The association of...,dataset,https://doi.org/10.7910/DVN/BFXCAM,doi:10.7910/DVN/BFXCAM,Base data for publication - The association of...,2020-06-15 18:12:08,The association of Coronavirus Disease-19 mort...,"Nag, Swapnika; Puri, Ankur; Brooks, Nathan; Ga...",covid19bcgML,The association of Coronavirus Disease-19 mort...,"Nag, Swapnika; Puri, Ankur; Brooks, Nathan; Ga...",s3://10.7910/DVN/BFXCAM,None,"[Medicine, Health and Life Sciences]",1.0,197619.0,RELEASED,1.0,0.0,2020-06-13T07:24:16Z,2020-06-15T18:12:08Z,"[{'name': 'Nag, Swapnika', 'affiliation': ''},...",None,"[Nag, Swapnika, Puri, Ankur

In [12]:
len(df_use)

269

In [13]:
#for matching other notebooks, rename 'df_use' back to 'df'
df = df_use

## 1. How many total objects (not just records) are in our main dataset extracts for each repository?
**Property:** unique_identifier

In [14]:
ids = df.DataverseDatasetsCrosswalk.unique_identifier
ids

0      doi:10.7910/DVN/N4VLQL
1      doi:10.7910/DVN/JNGTDF
2      doi:10.7910/DVN/RBZONT
3      doi:10.7910/DVN/BFXCAM
4      doi:10.7910/DVN/1YRRAC
                ...          
305       doi:10.18710/SSA38J
306       doi:10.34894/ITWFZP
312      doi:10.5064/F626JGPB
313       doi:10.18710/7JWAZX
314       doi:10.18710/OYIH83
Name: global_id, Length: 269, dtype: object

In [15]:
ids.nunique()

269

In [16]:
print(f'There are {len(ids)} items in the Dataverse Datasets extract of interest, with {ids.nunique()} unique IDs.')

There are 269 items in the Dataverse Datasets extract of interest, with 269 unique IDs.


## 2. See the "Licenses offered" tab in /Working documents/Licenses sheet for list of licenses by repo.

## Given the type(s) of license(s) offered by the repo, how many of each type is assigned?
**Property:** License

In [17]:
licenses = df.DataverseDatasetsCrosswalk.license
licenses

0      \n\n\n\n                               Waiver\...
1      \n\n\n\n                               Waiver\...
2      \n\n\n\n                               Waiver\...
3      \n\n\n\n                               Waiver\...
4      \n\n\n\n                               Waiver\...
                             ...                        
305    \n\n\n\n                               Waiver\...
306    \n\n\n\n                               Waiver\...
312    \n\n\n\n                               Waiver\...
313    \n\n\n\n                               Waiver\...
314    \n\n\n\n                               Waiver\...
Name: terms_of_use, Length: 269, dtype: object

In [18]:
# remove one NoneType row
licenses_clean = pd.Series(filter(None, licenses))
licenses_clean

0      \n\n\n\n                               Waiver\...
1      \n\n\n\n                               Waiver\...
2      \n\n\n\n                               Waiver\...
3      \n\n\n\n                               Waiver\...
4      \n\n\n\n                               Waiver\...
                             ...                        
264    \n\n\n\n                               Waiver\...
265    \n\n\n\n                               Waiver\...
266    \n\n\n\n                               Waiver\...
267    \n\n\n\n                               Waiver\...
268    \n\n\n\n                               Waiver\...
Length: 269, dtype: object

Since all the licenses have lots of whitespace and newline characters, we need to get rid of them before analysing

In [19]:
# Remove additional spaces and get rid of disclaimer
licenses_clean = licenses_clean.apply(
    lambda s: re.sub(
        'Waiver.*Dataverse\.',
        '',
        re.sub('(\s)+', ' ', s)
    )
)

In [20]:
licenses_clean

0                      CC0 - "Public Domain Dedication" 
1                      CC0 - "Public Domain Dedication" 
2                      CC0 - "Public Domain Dedication" 
3                      CC0 - "Public Domain Dedication" 
4                      CC0 - "Public Domain Dedication" 
                             ...                        
264      No waiver has been selected for this dataset...
265      No waiver has been selected for this dataset...
266     Waiver The Community Norms as well as good sc...
267                    CC0 - "Public Domain Dedication" 
268                    CC0 - "Public Domain Dedication" 
Length: 269, dtype: object

In [21]:
#group all CC0 licenses
licenses_clean.apply(
    lambda s: 'CC0' if 'CC0' in s else s
).value_counts()

CC0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

## 3. What is the mean number of characters (excluding whitespaces, if possible) per object?
**Property:** Description
**Related function:** `mean_characters`

In [22]:
licenses_clean.value_counts()

  CC0 - "Public Domain Dedication"                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [23]:
description = df.DataverseDatasetsCrosswalk.description
description

0      Here you can find pre-trained machine learning...
1      Here you can find GEOM, a dataset with over 33...
2      Data and code for replication of the Harvard D...
3      Base data for publication - The association of...
4                             Machine learn for glaucoma
                             ...                        
305    Results from a study of automatic aurora class...
306    Visual mental imagery is the quasi-perceptual ...
312    Overview Why do governments tolerate the viola...
313    Manually annotated 3-digit occupational codes ...
314    Manually annotated 3-digit occupation codes fr...
Name: description, Length: 269, dtype: object

In [24]:
print(f'Number of null descriptions: {sum(description.isna())}')

Number of null descriptions: 0


In [25]:
#remove None and drop duplicates
description_unique = pd.Series(filter(None, description)).drop_duplicates()
description_unique

0      Here you can find pre-trained machine learning...
1      Here you can find GEOM, a dataset with over 33...
2      Data and code for replication of the Harvard D...
3      Base data for publication - The association of...
4                             Machine learn for glaucoma
                             ...                        
264    Results from a study of automatic aurora class...
265    Visual mental imagery is the quasi-perceptual ...
266    Overview Why do governments tolerate the viola...
267    Manually annotated 3-digit occupational codes ...
268    Manually annotated 3-digit occupation codes fr...
Length: 266, dtype: object

In [26]:
print(f'{analysis.mean_characters(description_unique)} mean characters')

778.578947368421 mean characters


## 4. What is the mean number of characters (excluding whitespaces, if possible) per object?
**Property:** Methods
**Related function:** `mean_characters`

In [27]:
methods = df.DataverseDatasetsCrosswalk.methods
methods

In [28]:
#confirm missing for this repo
print(df.DataverseDatasetsCrosswalk.methods)

None


## 5. What are the min and max publication dates for each repo?

## How many objects were published each year for each repo?
**Property:** Publication date

In [29]:
publication_dates = df.DataverseDatasetsCrosswalk.publication_date
publication_dates

0     2021-02-11 22:02:13
1     2021-02-01 19:44:41
2     2021-10-26 16:11:34
3     2020-06-15 18:12:08
4     2018-11-15 07:21:36
              ...        
305   2021-10-22 00:00:04
306   2021-09-22 00:00:10
312   2021-04-21 19:53:12
313   2021-10-22 00:00:04
314   2021-10-22 00:00:04
Name: published_at, Length: 269, dtype: datetime64[ns]

Since the publication dates are already datetime objects, we can simply query them directly to get the desired metrics.

In [30]:
publication_dates.min(), publication_dates.max()

(Timestamp('2008-12-19 00:00:00'), Timestamp('2021-12-02 09:25:43'))

The following takes the `year` attribute of the dates and counts the number of occurrences. These counts are then sorted by the index (year).

In [31]:
publication_dates.apply(lambda date: date.year).value_counts().sort_index()

2008     1
2011     1
2012     2
2013     1
2014     3
2015    11
2016    14
2017    11
2018    32
2019    37
2020    65
2021    91
Name: published_at, dtype: int64

In [32]:
#ready for plotting export - order and make dataframe
pub_dates_export = publication_dates.apply(lambda date: date.year).value_counts().sort_index().to_frame()

In [33]:
pub_dates_export

,published_at
2008,1
2011,1
2012,2
2013,1
2014,3
2015,11
2016,14
2017,11
2018,32
2019,37


In [34]:
#update column names
pub_dates_export_ready = pub_dates_export.reset_index(level=0)
pub_dates_export_ready.columns = ['year', 'count']
pub_dates_export_ready

,year,count
0,2008,1
1,2011,1
2,2012,2
3,2013,1
4,2014,3
5,2015,11
6,2016,14
7,2017,11
8,2018,32
9,2019,37


In [35]:
#add column with name of repo
pub_dates_export_ready['repo'] = 'dataverse_datasets'

In [36]:
pub_dates_export_ready

,year,count,repo
0,2008,1,dataverse_datasets
1,2011,1,dataverse_datasets
2,2012,2,dataverse_datasets
3,2013,1,dataverse_datasets
4,2014,3,dataverse_datasets
5,2015,11,dataverse_datasets
6,2016,14,dataverse_datasets
7,2017,11,dataverse_datasets
8,2018,32,dataverse_datasets
9,2019,37,dataverse_datasets


In [37]:
#export to Figures folder
pub_dates_export_ready.to_csv('..\\..\\Figures\\Figure1\\repository_dates\\dataverse_datasets_pub_years.csv')

## 6. What are the unweighted mean, median, and max file sizes among all ingested files?
**Property:** File size
**Related function:** `get_summary_statistics`

We first get the file size attribute using the crosswalk.

In [38]:
file_size = df.DataverseDatasetsCrosswalk.file_size
file_size

0      [23692, 23692, 6394678, 5171913, 5273418, 3033...
1      [42687717910, 10138004284, 1025881930, 1317167...
2                         [300383, 13952, 727210, 47666]
3                                                [18293]
4                                            [123981705]
                             ...                        
305    [4709, 8761, 76439416, 161297, 10646108, 37562...
306    [14282423, 191284518, 2595891813, 3640350603, ...
312    [218160, 1564, 229580, 19181, 210157, 7623, 10...
313                     [1456, 336481763, 348128, 36128]
314                                    [1416, 456047564]
Name: files, Length: 269, dtype: object

To extract the data, we use a list comprehension the takes the size list from each index of the Series
```python
for size_list in dd_file_size
```
and then take each of the values from the size lists
```python
for size in size_list
```

This list comprehension is then passed into numpy for easier access to the summary statistic functions.

**Note:** Since the following cell performs an "in-place" operation (overwrites the Series stored in `file_size`), running the cell multiple times will result in an error. If this occurs, simply re-run the above cell that defiens `file_size` and then re-run the cell below.

In [39]:
dd_file_size = np.array([size for size_list in file_size for size in size_list])
dd_file_size

array([    23692,     23692,   6394678, ...,     36128,      1416,
       456047564], dtype=int64)

Now we can easily summarize the data.

In [40]:
analysis.get_summary_statistics(dd_file_size, suppress_output=False);

mean: 132906340.10713574
median: 67310.0
max: 61227536790


## 7. What are the mean, median, and max number of files per object?
**Property:** URL
**Related function:** `get_summary_statistics`

In [41]:
files = df.DataverseDatasetsCrosswalk.url
files

0      [https://dataverse.harvard.edu/api/access/data...
1      [https://dataverse.harvard.edu/api/access/data...
2                               [None, None, None, None]
3      [https://dataverse.harvard.edu/api/access/data...
4      [https://dataverse.harvard.edu/api/access/data...
                             ...                        
305    [https://dataverse.no/api/access/datafile/1893...
306    [None, None, None, None, None, None, None, Non...
312    [None, None, None, None, None, None, None, Non...
313    [https://dataverse.no/api/access/datafile/1001...
314    [https://dataverse.no/api/access/datafile/7957...
Name: files, Length: 269, dtype: object

In [42]:
#remove None values from lists
files_clean = files
files_clean = files.apply(lambda el: [x for x in el if x is not None])
files_clean

0      [https://dataverse.harvard.edu/api/access/data...
1      [https://dataverse.harvard.edu/api/access/data...
2                                                     []
3      [https://dataverse.harvard.edu/api/access/data...
4      [https://dataverse.harvard.edu/api/access/data...
                             ...                        
305    [https://dataverse.no/api/access/datafile/1893...
306                                                   []
312                                                   []
313    [https://dataverse.no/api/access/datafile/1001...
314    [https://dataverse.no/api/access/datafile/7957...
Name: files, Length: 269, dtype: object

In [43]:
files_counts = files_clean.apply(len)
files_counts

0      2496
1         6
2         0
3         1
4         1
       ... 
305       6
306       0
312       0
313       4
314       2
Name: files, Length: 269, dtype: int64

In [44]:
#confirm very large number of files for first object is accurate
files_clean[0]

['https://dataverse.harvard.edu/api/access/datafile/4377731',
 'https://dataverse.harvard.edu/api/access/datafile/4376541',
 'https://dataverse.harvard.edu/api/access/datafile/4376504',
 'https://dataverse.harvard.edu/api/access/datafile/4376512',
 'https://dataverse.harvard.edu/api/access/datafile/4376519',
 'https://dataverse.harvard.edu/api/access/datafile/4377789',
 'https://dataverse.harvard.edu/api/access/datafile/4376533',
 'https://dataverse.harvard.edu/api/access/datafile/4362822',
 'https://dataverse.harvard.edu/api/access/datafile/4364125',
 'https://dataverse.harvard.edu/api/access/datafile/4365287',
 'https://dataverse.harvard.edu/api/access/datafile/4374074',
 'https://dataverse.harvard.edu/api/access/datafile/4373419',
 'https://dataverse.harvard.edu/api/access/datafile/4364159',
 'https://dataverse.harvard.edu/api/access/datafile/4373904',
 'https://dataverse.harvard.edu/api/access/datafile/4364067',
 'https://dataverse.harvard.edu/api/access/datafile/4364605',
 'https:

In [45]:
analysis.get_summary_statistics(files_counts, suppress_output=False);

mean: 50.394052044609666
median: 4.0
max: 2496


## 8. What are the mean, median, and max total dataset size (summed across all files) per object?
**Property:** Dataset size
**Related function:** `get_summary_statistics`

Below, we see that the `dataset_size` property returns the same data as the `file_size` property, so we again retrieve the `contentSize` column of the returned DataFrame.

In [46]:
dataset_size = df.DataverseDatasetsCrosswalk.dataset_size
dataset_size

0      [23692, 23692, 6394678, 5171913, 5273418, 3033...
1      [42687717910, 10138004284, 1025881930, 1317167...
2                         [300383, 13952, 727210, 47666]
3                                                [18293]
4                                            [123981705]
                             ...                        
305    [4709, 8761, 76439416, 161297, 10646108, 37562...
306    [14282423, 191284518, 2595891813, 3640350603, ...
312    [218160, 1564, 229580, 19181, 210157, 7623, 10...
313                     [1456, 336481763, 348128, 36128]
314                                    [1416, 456047564]
Name: files, Length: 269, dtype: object

Since we now want to look at the summary statistics for each dataset (all files for a given object), we first sum the values in the lists above before applying analyses. Again, the following cell will overwrite the `dataset_size` variable given above.

In [47]:
dataset_size = dataset_size.apply(sum)
dataset_size

0      661975747851
1      104120904447
2           1089211
3             18293
4         123981705
           ...     
305       124822703
306     36997808299
312       662353163
313       336867475
314       456048980
Name: files, Length: 269, dtype: int64

In [48]:
analysis.get_summary_statistics(dataset_size, suppress_output=False);

mean: 7443249121.613383
median: 119064266.0
max: 661975747851


## 9. How many of each scientific domain are assigned?
**Property:** Domain

In [49]:
domains = df.DataverseDatasetsCrosswalk.domain
domains

0          [Chemistry, Computer and Information Science]
1          [Chemistry, Computer and Information Science]
2      [Medicine, Health and Life Sciences, Social Sc...
3                   [Medicine, Health and Life Sciences]
4                   [Medicine, Health and Life Sciences]
                             ...                        
305    [Computer and Information Science, Mathematica...
306                                    [Social Sciences]
312                                    [Social Sciences]
313                   [Computer and Information Science]
314                   [Computer and Information Science]
Name: subjects, Length: 269, dtype: object

Since the Series contain lists of the related scientific domains, we can use the following code to break this down into a single-entry Series of the domains.

**Explanation**
```python
dd_domains.apply(lambda li: pd.Series(li, dtype=str))
```
This is used to turn each domain list into a Series of type str, which overall transforms the Series of lists into a DataFrame of strings.

```python
dd_domains.apply(lambda li: pd.Series(li, dtype=str)).stack()
````
This takes the DataFrame and transforms it into a stacked Series containing all cells of the DataFrame.

```python
dd_domains.apply(lambda li: pd.Series(li, dtype=str)).stack().reset_index(drop=True)
```
This resets the multi-index of the Series (which is present to represent the row and column labels of the original DataFrame) into a single sequential index.

#### note: used same approach, but not function series_list_to_series on some other notebooks
eventually need to standardize

In [50]:
domains_all = analysis.series_list_to_series(domains, dtype=str)
domains_all

0                               Chemistry
1        Computer and Information Science
2                               Chemistry
3        Computer and Information Science
4      Medicine, Health and Life Sciences
                      ...                
377                               Physics
378                       Social Sciences
379                       Social Sciences
380      Computer and Information Science
381      Computer and Information Science
Length: 382, dtype: object

In [51]:
domain_counts = domains_all.value_counts().to_frame()
domain_counts['percent'] = domain_counts/len(domains) * 100
domain_counts

,0,percent
Social Sciences,110,40.892193
Computer and Information Science,97,36.059480
"Medicine, Health and Life Sciences",43,15.985130
Earth and Environmental Sciences,29,10.780669
Physics,17,6.319703
Engineering,16,5.947955
Arts and Humanities,15,5.576208
Agricultural Sciences,13,4.832714
Other,11,4.089219
Chemistry,10,3.717472


## 10. What is the mean number of characters (excluding whitespaces, if possible) per object?
**Property:** Technical details
**Related function:** `mean_characters`

In [52]:
# "usage notes" is not in crosswalk - need to return to

## 11-13. What are the mean and median total number of keyword terms per object, after merging results for Keyword, Geographic keyword, and Scientific keyword?
**Property:** Keyword

In [53]:
print(df.DataverseDatasetsCrosswalk.keyword)

0      [machine learning, computational drug discover...
1      [computational chemistry, quantum chemistry, m...
2                                                   None
3                                                   None
4                                                   None
                             ...                        
305    [auroral images, auroral classification, convo...
306    [Visual mental imagery, fMRI at 7T, Retinotopi...
312    [law enforcement, housing, poverty programs, i...
313    [Training dataset, Digit codes, Population cen...
314    [Training dataset, Digit codes, Population cen...
Name: keywords, Length: 269, dtype: object


In [54]:
print(df.DataverseDatasetsCrosswalk.geographic_keyword)

0                                                   None
1                                                   None
2                                                   None
3                                                   None
4                                                   None
                             ...                        
305           [{'country': 'Sweden', 'city': 'Kiruna,'}]
306                                                 None
312    [{'country': 'Peru', 'city': 'Lima,'}, {'other...
313                                                 None
314                                                 None
Name: geographicCoverage, Length: 269, dtype: object


In [55]:
print(df.DataverseDatasetsCrosswalk.scientific_keyword)

None


In [56]:
#create and concatenate keywords
keywords1 = df.DataverseDatasetsCrosswalk.keyword
keywords2 = df.DataverseDatasetsCrosswalk.geographic_keyword

In [57]:
keywords_all = pd.concat([keywords1, keywords2], axis = 1)
keywords_all

,keywords,geographicCoverage
0,"[machine learning, computational drug discover...",None
1,"[computational chemistry, quantum chemistry, m...",None
2,None,None
3,None,None
4,None,None
...,...,...
305,"[auroral images, auroral classification, convo...","[{'country': 'Sweden', 'city': 'Kiruna,'}]"
306,"[Visual mental imagery, fMRI at 7T, Retinotopi...",None
312,"[law enforcement, housing, poverty programs, i...","[{'country': 'Peru', 'city': 'Lima,'}, {'other..."
313,"[Training dataset, Digit codes, Population cen...",None


Since the `geographicCoverage` values are lists of dictionaries where the values represent the words that we want to count, we need to modify the column to extract these. But first, we want to replace the `None` values with empty lists so the count of string values evaluates to 0.

In [58]:
#replace None with empty list
keywords_all = keywords_all.apply(
    lambda row: row.apply(
        lambda cell: cell if cell else []
    ),
    axis=1
)

In [59]:
keywords_all

,keywords,geographicCoverage
0,"[machine learning, computational drug discover...",[]
1,"[computational chemistry, quantum chemistry, m...",[]
2,[],[]
3,[],[]
4,[],[]
...,...,...
305,"[auroral images, auroral classification, convo...","[{'country': 'Sweden', 'city': 'Kiruna,'}]"
306,"[Visual mental imagery, fMRI at 7T, Retinotopi...",[]
312,"[law enforcement, housing, poverty programs, i...","[{'country': 'Peru', 'city': 'Lima,'}, {'other..."
313,"[Training dataset, Digit codes, Population cen...",[]


In [60]:
#extract values from dictionaries in `geographicCoverage`
keywords_all['geographicCoverage'] = keywords_all['geographicCoverage'].apply(
    lambda entry: [' '.join(list(inner_dict.values())) for inner_dict in entry]
)

In [61]:
keywords_all

,keywords,geographicCoverage
0,"[machine learning, computational drug discover...",[]
1,"[computational chemistry, quantum chemistry, m...",[]
2,[],[]
3,[],[]
4,[],[]
...,...,...
305,"[auroral images, auroral classification, convo...","[Sweden Kiruna,]"
306,"[Visual mental imagery, fMRI at 7T, Retinotopi...",[]
312,"[law enforcement, housing, poverty programs, i...","[Peru Lima,, Latin America,, Colombia Bogotá,,..."
313,"[Training dataset, Digit codes, Population cen...",[]


In [62]:
#count up keywords
keyword_counts = keywords_all.apply(
    lambda row: sum([analysis.count_words(row[column]) for column in keywords_all.columns]),
    axis=1
)
keyword_counts

0       7
1      10
2       0
3       0
4       0
       ..
305    13
306    10
312    23
313     8
314     8
Length: 269, dtype: int64

In [63]:
analysis.get_summary_statistics(keyword_counts)

{'mean': 5.613382899628252, 'median': 5.0, 'max': 102}

## 14. Who are the most common funding agencies for each repo? What are the object counts per agency?
**Property:** Funding agency

In [64]:
funders = df.DataverseDatasetsCrosswalk.funding_agency
funders

0                                                   None
1                                                   None
2                                                   None
3                                                   None
4                                                   None
                             ...                        
305                                                 None
306                                                 None
312    National Science Foundation: DEG 0644491\n    ...
313                                                 None
314                                                 None
Name: grant_info, Length: 269, dtype: object

In [65]:
#how many are not none?
len(pd.Series(filter(None, funders)))

14

In [66]:
funders.value_counts().to_frame()

,grant_info
"U.S. Department of Energy, Office of Science, Basic Energy Sciences: DE-SC0012573\n",3
SSHRC: 430-2018-1069\n FQRSC: 253243\n National Science Foundation: DMS-1737865\n University of Delaware\n,1
DFG: FA 211/23-1\n,1
PDEL\n LACEEP\n,1
"International Initiative for Impact Evaluation (3ie): TW13-1038\n Natural Environment Research Council, NERC: NE/R014094/1\n",1
United States Agency for International Development - USAID\n,1
"Department of Science and Technology, Govt. of India: EMR/2016/006183\n",1
"IRT M2P, Ressources Emergentes (RESEM)\n",1
The Bill & Melinda Gates Foundation\n,1
European Research Council: 804233\n Research Council of Norway: 288565\n Northern Norway Regional Health Authority: HNF1449-19\n UiT’s strategic funding program\n UiT Tematiske Satsinger\n,1


## 15. What are the mean, median, and max number of Views per object?
**Property:** Views
**Related function:** `get_summary_statistics`

In [67]:
views = df.DataverseDatasetsCrosswalk.views
views

In [68]:
#confirm missing for this repo
print(df.DataverseDatasetsCrosswalk.views)

None


## 16. What are the mean, median, and max (total) number of downloads per object?
**Property:** Downloads
**Related function:** `get_summary_statistics`

In [69]:
downloads = df.DataverseDatasetsCrosswalk.downloads
downloads

0       19.0
1        2.0
2        0.0
3       29.0
4      892.0
       ...  
305    724.0
306      0.0
312      1.0
313     19.0
314     94.0
Name: num_downloads, Length: 269, dtype: float64

In [70]:
#get summary statistics
analysis.get_summary_statistics(downloads)

{'mean': 123.48327137546468, 'median': 32.0, 'max': 992.0}

## 17. What are the mean, median, and max Citation counts per object?
**Property:** Citation count
**Related function:** `get_summary_statistics`

In [71]:
citation_count = df.DataverseDatasetsCrosswalk.citation_count
citation_count

In [72]:
#confirm missing for this repo
print(df.DataverseDatasetsCrosswalk.citation_count)

None


## 18. How many objects contain each given resource type?
**Property:** Resource type

In [73]:
resource_types = df.DataverseDatasetsCrosswalk.resource_type
resource_types

0           None
1           None
2           None
3           None
4           None
         ...    
305       Images
306    fMRI data
312         None
313         None
314         None
Name: kind_of_data, Length: 269, dtype: object

In [74]:
#how many are not none?
len(pd.Series(filter(None, resource_types)))

37

In [75]:
resource_counts = resource_types.value_counts().to_frame()
resource_counts['percent'] = resource_counts['kind_of_data']/len(resource_types) * 100
resource_counts

,kind_of_data,percent
Experimental data,3,1.115242
xls\n,2,0.743494
Images,2,0.743494
Dissertation Replication,1,0.371747
GeoTIFFs spatial data,1,0.371747
Web scraping,1,0.371747
clinical data; observation data/ratings,1,0.371747
"Numpy zip, csv",1,0.371747
Training logs of medium to long distance runners,1,0.371747
Geographic Information Systems (GIS) data,1,0.371747


## 19. How many objects contain each type of file extension given?
**Property:** File Extension

In [76]:
files = df.DataverseDatasetsCrosswalk.file_extension
files

0      [all.tab, all.tab, all.tab, all.tab, all.tab, ...
1      [drugs_crude.msgpack.tar.gz, drugs_featurized....
2      [Codebook 4_3.tab, HDSR_social_distancing.R, s...
3                             [Consolidated data_vF.tab]
4                                   [processed_data.zip]
                             ...                        
305    [00_ReadMe.txt, Auroral Image Classification w...
306    [behav.zip, S01add_ana.zip, S01functional_AP_P...
312    [Holland_HousingSurvey_English.pdf, Holland_In...
313    [00_README.txt, RandomSample_trainingset.zip, ...
314             [00_Read_Me.txt, 3DigitsTrainingset.zip]
Name: files, Length: 269, dtype: object

The following code separates the full file extensions (all dot-separated values after the first dot) for a list of files and creates a set, allowing us to only look at the number of objects that contain a given extension.

In [77]:
file_extension_set = files.apply(
    lambda file_list: list({''.join(Path(file).suffixes) for file in file_list})
)
file_extension_set

0      [, .tab, .pickle, .txt, .json, .md, .zip, .log...
1                        [.md, .msgpack.tar.gz, .tar.gz]
2                                      [.tab, .R, .docx]
3                                                 [.tab]
4                                                 [.zip]
                             ...                        
305                     [.pdf, .ipynb, .txt, .zip, .npz]
306                                               [.zip]
312                            [.pdf, .tab, .txt, .xlsx]
313                                   [.zip, .npy, .txt]
314                                         [.zip, .txt]
Name: files, Length: 269, dtype: object

In [78]:
#expand all, keeping index to group at object level
file_extension_set = file_extension_set.explode()
file_extension_set

0             
0         .tab
0      .pickle
0         .txt
0        .json
        ...   
313       .zip
313       .npy
313       .txt
314       .zip
314       .txt
Name: files, Length: 1581, dtype: object

In [79]:
#this needs clean up - some are named with multiple . which reads as file extension from above code
file_extension_set.unique()

array(['', '.tab', '.pickle', '.txt', '.json', '.md', '.zip', '.log',
       '.pth.tar', '.npz', '.msgpack.tar.gz', '.tar.gz', '.R', '.docx',
       '.xls', '.tar.xz', '.spv', '.csaplan', '.py', '.png', '.csv',
       '.xlsx', '.pdf', '.key', '.dat', '.Rd', '.stan', '.dta', '.Rproj',
       '.yml', '.cpickle', '.nc-a', '.nc-c', '.nc-b', '.pkl', '.rar',
       '.parquet', '.ipynb', '.Rmd', '.str', '.fasta', '.loci',
       '.alata_reordered.str', '.35', '.out', '.xyz', '.smcl', '.do',
       '.csv.tab', '.RData', '.1.tab', '.ods', '.joblib', '.nc', '.h5',
       '.fits', '.eps', '.mk', '.ttf', '.sh', '.npy', '.POST_STACK.segy',
       '.rtf', '.R.txt', '.vec', '.tac', '.chi', '.prj', '.zip.001',
       '.zip.002', '.rdata', '.3-5.LPJ-GUESS.zip', '.2.ML.zip',
       '.3-5.ML.zip', '.1.PMIP_SAT_spread.zip', '.fdt', '.set', '.rds',
       '.9.zip.rar', '.zip.rar', '.0.zip.rar', '.gp', '.mff', '.awk',
       '.in', '.rd', '.Rda', '.tiff', '.fasta.gz', '.hhm.gz', '.tif',
       '.db', '.sql'

In [80]:
#check an ESTIMATE of most common file extensions - needs additional clean up
file_ext_counts = file_extension_set.value_counts().to_frame()
file_ext_counts['percent'] = file_ext_counts['files']/len(files) * 100
file_ext_counts

,files,percent
.tab,96,35.687732
.txt,68,25.278810
.csv,54,20.074349
.pdf,52,19.330855
.R,50,18.587361
...,...,...
.tree.sp_eu.forest00k.tamarix.spp._p_1km_s0..0cm_2000..2017_v0.1.tif,1,0.371747
.v.nov_d_1km_s0..0cm_2014..2017_v0.1.tif,1,0.371747
.tree.sp_eu.forest00k.betula.spp._p_1km_s0..0cm_2000..2017_v0.1.tif,1,0.371747
.tree.sp_eu.forest00k.populus.nigra_p_1km_s0..0cm_2000..2017_v0.1.tif,1,0.371747


In [81]:
#confirm accuracy
#expecting 18 objects with .md files
file_ext_counts.head(10)

,files,percent
.tab,96,35.687732
.txt,68,25.278810
.csv,54,20.074349
.pdf,52,19.330855
.R,50,18.587361
.zip,43,15.985130
,25,9.293680
.py,21,7.806691
.md,18,6.691450
.docx,17,6.319703


In [82]:
#check objects with .md files
file_extension_set[file_extension_set == '.md']

0      .md
1      .md
13     .md
28     .md
44     .md
65     .md
87     .md
93     .md
94     .md
95     .md
98     .md
105    .md
132    .md
140    .md
148    .md
189    .md
201    .md
230    .md
Name: files, dtype: object

In [83]:
#confirmed - this is counting at object level
len(file_extension_set[file_extension_set == '.md'])

18

In [84]:
#file_ext_counts['percent'] = round(file_ext_counts['files']/len(files) * 100)
#file_ext_counts[file_ext_counts['percent'] >= 5]

In [85]:
#export for further clean up, refining estimates, and plotting
files_exp_ready = file_extension_set.to_frame()
files_exp_ready

,files
0,
0,.tab
0,.pickle
0,.txt
0,.json
...,...
313,.zip
313,.npy
313,.txt
314,.zip


In [86]:
#reset index and update column names
files_exp_ready = files_exp_ready.reset_index(level=0)
files_exp_ready

,index,files
0,0,
1,0,.tab
2,0,.pickle
3,0,.txt
4,0,.json
...,...,...
1576,313,.zip
1577,313,.npy
1578,313,.txt
1579,314,.zip


In [87]:
files_exp_ready.columns = ['index', 'files']

#add column with name of repo
files_exp_ready['repo'] = 'dataverse_datasets'

files_exp_ready

,index,files,repo
0,0,,dataverse_datasets
1,0,.tab,dataverse_datasets
2,0,.pickle,dataverse_datasets
3,0,.txt,dataverse_datasets
4,0,.json,dataverse_datasets
...,...,...,...
1576,313,.zip,dataverse_datasets
1577,313,.npy,dataverse_datasets
1578,313,.txt,dataverse_datasets
1579,314,.zip,dataverse_datasets


In [88]:
#export to Figures folder
files_exp_ready.to_csv('..\\..\\Figures\\Figure2\\file_ext_data\\dataverse_datasets_extensions.csv')

## 19.5 How many files of each type of file extension are present?
**Property:** File extension

In [89]:
files = df.DataverseDatasetsCrosswalk.file_extension
files

0      [all.tab, all.tab, all.tab, all.tab, all.tab, ...
1      [drugs_crude.msgpack.tar.gz, drugs_featurized....
2      [Codebook 4_3.tab, HDSR_social_distancing.R, s...
3                             [Consolidated data_vF.tab]
4                                   [processed_data.zip]
                             ...                        
305    [00_ReadMe.txt, Auroral Image Classification w...
306    [behav.zip, S01add_ana.zip, S01functional_AP_P...
312    [Holland_HousingSurvey_English.pdf, Holland_In...
313    [00_README.txt, RandomSample_trainingset.zip, ...
314             [00_Read_Me.txt, 3DigitsTrainingset.zip]
Name: files, Length: 269, dtype: object

We want to flatten the series of lists to a series containing all of the values, and then take the extension from there

In [90]:
#expand lists to each item is alone
files = analysis.series_list_to_series(files, dtype=str)
files

0                             all.tab
1                             all.tab
2                             all.tab
3                             all.tab
4                             all.tab
                     ...             
15060    RandomSample_trainingset.zip
15061                     x_valid.npy
15062                     y_valid.npy
15063                  00_Read_Me.txt
15064          3DigitsTrainingset.zip
Length: 15065, dtype: object

In [91]:
#extract file extensions
files_ext = files.apply(lambda fn: Path(fn).suffixes)
files_ext

0        [.tab]
1        [.tab]
2        [.tab]
3        [.tab]
4        [.tab]
          ...  
15060    [.zip]
15061    [.npy]
15062    [.npy]
15063    [.txt]
15064    [.zip]
Length: 15065, dtype: object

In [92]:
#count up total files
files_totals = files_ext.value_counts().to_frame()
files_totals

,0
[.tab],2953
[.xyz],2310
[],962
[.Rdata],633
[.json],523
...,...
"[.v, .feb_u, .975_1km_s0, ., .0cm_2014, ., .2017_v0, .1, .tif]",1
"[.v, .jan_d_1km_s0, ., .0cm_2014, ., .2017_v0, .1, .tif]",1
"[.v, .jan_sd_1km_s0, ., .0cm_2014, ., .2017_v0, .1, .tif]",1
"[.v, .jan_sd, .u, .975_1km_s0, ., .0cm_2014, ., .2017_v0, .1, .tif]",1


We can see that there are quite a few "strange" results here, frequently called by user's using periods in their file names, as well as some cases where files don't necessarily have a file extension.

**Note:** The case of `gitignore` comes from the `.gitignore` file for a git repository


In [93]:
files_totals.head(10)

,0
[.tab],2953
[.xyz],2310
[],962
[.Rdata],633
[.json],523
[.pdf],505
[.R],447
[.csv],430
[.zip],363
[.mat],362


## 20. How many objects contain each type of File format given?
**Property:** File format

In [94]:
file_formats = df.DataverseDatasetsCrosswalk.file_format
file_formats

0      [text/tab-separated-values, text/tab-separated...
1      [application/x-compressed-tar, application/x-c...
2      [text/tab-separated-values, type/x-r-syntax, t...
3                            [text/tab-separated-values]
4                                      [application/zip]
                             ...                        
305    [text/plain, application/x-ipynb+json, applica...
306    [application/zip, application/zip, application...
312    [application/pdf, text/tab-separated-values, a...
313    [text/plain, application/zip, application/octe...
314                        [text/plain, application/zip]
Name: files, Length: 269, dtype: object

In [95]:
#since interested in per object, need to group by ID
formats_ids = pd.concat([ids, file_formats], axis = 1)
formats_ids

,global_id,files
0,doi:10.7910/DVN/N4VLQL,"[text/tab-separated-values, text/tab-separated..."
1,doi:10.7910/DVN/JNGTDF,"[application/x-compressed-tar, application/x-c..."
2,doi:10.7910/DVN/RBZONT,"[text/tab-separated-values, type/x-r-syntax, t..."
3,doi:10.7910/DVN/BFXCAM,[text/tab-separated-values]
4,doi:10.7910/DVN/1YRRAC,[application/zip]
...,...,...
305,doi:10.18710/SSA38J,"[text/plain, application/x-ipynb+json, applica..."
306,doi:10.34894/ITWFZP,"[application/zip, application/zip, application..."
312,doi:10.5064/F626JGPB,"[application/pdf, text/tab-separated-values, a..."
313,doi:10.18710/7JWAZX,"[text/plain, application/zip, application/octe..."


In [96]:
#expand so each file format is own row
#drop duplicates since interested in format at level of object, not how many of each format for each object
formats_ids_all = formats_ids.explode('files').drop_duplicates()
formats_ids_all

,global_id,files
0,doi:10.7910/DVN/N4VLQL,text/tab-separated-values
0,doi:10.7910/DVN/N4VLQL,application/json
0,doi:10.7910/DVN/N4VLQL,application/octet-stream
0,doi:10.7910/DVN/N4VLQL,application/zip
0,doi:10.7910/DVN/N4VLQL,text/plain
...,...,...
313,doi:10.18710/7JWAZX,text/plain
313,doi:10.18710/7JWAZX,application/zip
313,doi:10.18710/7JWAZX,application/octet-stream
314,doi:10.18710/OYIH83,text/plain


In [97]:
#group by file format  to find number of objects
formats_calcs = formats_ids_all.groupby('files').value_counts().groupby('files').size().sort_values(ascending = False)
formats_calcs = formats_calcs.to_frame()
formats_calcs['percent'] = formats_calcs[0]/len(file_formats) * 100
formats_calcs

,0,percent
files,,
text/tab-separated-values,100,37.174721
text/plain,67,24.907063
application/pdf,53,19.702602
text/csv,50,18.587361
application/octet-stream,50,18.587361
type/x-r-syntax,45,16.728625
application/zip,44,16.356877
text/plain; charset=US-ASCII,41,15.241636
application/gzip,19,7.063197


In [98]:
#confirm accuracy of approach
len(formats_ids_all[formats_ids_all['files'] == 'text/csv'])

50

## 21. How many objects contain each type of Media type given?
**Property:** Media type

In [99]:
media_types = df.DataverseDatasetsCrosswalk.media_type
media_types

In [100]:
#confirm missing for this repo
print(df.DataverseDatasetsCrosswalk.media_type)

None


## 22. a) How many objects report one related resource type, and b) how many objects report each of those types? c) How many objects report multiple related resource types (regardless of which types)?
**Property:** Related resource type

In [101]:
related_resource_types = df.DataverseDatasetsCrosswalk.related_resource_type
related_resource_types

In [102]:
#confirm missing for this repo
print(df.DataverseDatasetsCrosswalk.related_resource_type)

None



## 23-25. If there is an entry for an object in one of the three properties (Original data URL, Primary manuscript PID/URL, and Related resource identifier) count as Related resources = True and then count the number of objects that return True.
**Property:** Related Resource Identifier

In [103]:
related_resource1 = df.DataverseDatasetsCrosswalk.original_data_url
related_resource2 = df.DataverseDatasetsCrosswalk.primary_manuscript
related_resource3 = df.DataverseDatasetsCrosswalk.related_resource_identifier

In [104]:
print(related_resource1)

None


In [105]:
print(related_resource2)

0                     [https://arxiv.org/abs/2012.08452]
1                     [https://arxiv.org/abs/2006.05531]
2                                                 [None]
3                                                   None
4                                                   None
                             ...                        
305    [None, https://www.geosci-instrum-method-data-...
306         [https://doi.org/10.1007/s00429-019-01828-6]
312              [https://doi.org/10.1017/9781316795613]
313                                                 None
314                                                 None
Name: publications, Length: 269, dtype: object


In [106]:
print(related_resource3)

0      None
1      None
2      None
3      None
4      None
       ... 
305    None
306    None
312    None
313    None
314    None
Name: relatedMaterial, Length: 269, dtype: object


In [107]:
#concatenate related resources
related_resource_all = pd.concat([related_resource2, related_resource3], axis = 1)
related_resource_all

,publications,relatedMaterial
0,[https://arxiv.org/abs/2012.08452],None
1,[https://arxiv.org/abs/2006.05531],None
2,[None],None
3,None,None
4,None,None
...,...,...
305,"[None, https://www.geosci-instrum-method-data-...",None
306,[https://doi.org/10.1007/s00429-019-01828-6],None
312,[https://doi.org/10.1017/9781316795613],None
313,None,None


In [108]:
#replace None values with empty list
related_resource_clean = related_resource_all.apply(
    lambda row: row.apply(
        lambda cell: cell if cell else []
    ),
    axis=1
)

related_resource_clean

,publications,relatedMaterial
0,[https://arxiv.org/abs/2012.08452],[]
1,[https://arxiv.org/abs/2006.05531],[]
2,[None],[]
3,[],[]
4,[],[]
...,...,...
305,"[None, https://www.geosci-instrum-method-data-...",[]
306,[https://doi.org/10.1007/s00429-019-01828-6],[]
312,[https://doi.org/10.1017/9781316795613],[]
313,[],[]


There are `None` values in lists as `[None]` which need to be removed, as they are not true related resources.

In [109]:
#filter None values from lists
related_resource_clean['publications'] = related_resource_clean['publications'].apply(lambda x: list(filter(None, x)))
related_resource_clean['relatedMaterial'] = related_resource_clean['relatedMaterial'].apply(lambda x: list(filter(None, x)))

In [110]:
#confirm accuracy of replacement

In [111]:
related_resource_all.head(20)

,publications,relatedMaterial
0,[https://arxiv.org/abs/2012.08452],None
1,[https://arxiv.org/abs/2006.05531],None
2,[None],None
3,None,None
4,None,None
5,[http://faculty.smu.edu/tfomby/eco5385_eco6380...,None
7,None,None
8,[None],None
9,None,None
10,[],None


In [112]:
related_resource_clean.head(20)

,publications,relatedMaterial
0,[https://arxiv.org/abs/2012.08452],[]
1,[https://arxiv.org/abs/2006.05531],[]
2,[],[]
3,[],[]
4,[],[]
5,[http://faculty.smu.edu/tfomby/eco5385_eco6380...,[]
7,[],[]
8,[],[]
9,[],[]
10,[],[]


The following code looks at each row of the resulting DataFrame `related_resource_clean` and returns `True` if any of the values in it evaluate to `True` (ie not None or empty list)

To break it down,
```python
related_resources.apply(
    lambda row: ...,
    axis=1
)
```
applies the `lambda: row` function to every row in the DataFrame.

```python
lambda row: any(row)
```
is `True` if `any` of the values in the row evaluate as `True`.

In [113]:
related_resource_mask = related_resource_clean.apply(
    lambda row: any(row), 
    axis=1
)

related_resource_mask

0       True
1       True
2      False
3      False
4      False
       ...  
305     True
306     True
312     True
313    False
314    False
Length: 269, dtype: bool

In [114]:
print(f'{related_resource_mask.sum()} of the {len(related_resource_clean)} objects have a related_resource')

67 of the 269 objects have a related_resource


## 23-25. Also, what is the mean number of related resource links per object (again looking at the three properties (Original data URL, Primary manuscript PID/URL, nd Related resource identifier)?
**Property:** Related Resource Identifier

We want to calculate this value to be mean number of links *for objects that have links*

We can start by subsetting `related_resource_clean` by `related_resource_mask`

In [115]:
related_resource_clean.head()

,publications,relatedMaterial
0,[https://arxiv.org/abs/2012.08452],[]
1,[https://arxiv.org/abs/2006.05531],[]
2,[],[]
3,[],[]
4,[],[]


In [116]:
related_resource_mask

0       True
1       True
2      False
3      False
4      False
       ...  
305     True
306     True
312     True
313    False
314    False
Length: 269, dtype: bool

In [117]:
rr_subset = related_resource_clean[related_resource_mask]
rr_subset

,publications,relatedMaterial
0,[https://arxiv.org/abs/2012.08452],[]
1,[https://arxiv.org/abs/2006.05531],[]
5,[http://faculty.smu.edu/tfomby/eco5385_eco6380...,[]
19,[https://journals.plos.org/plosone/article?id=...,[]
26,[https://doi.org/10.7910/DVN/LD0ITE],[Enter any related materials here and create a...
29,[http://doi.org/10.1017/S0003055419000522],[]
30,[],[Enter any related materials here and create a...
31,[http://books.nips.cc/papers/files/nips25/NIPS...,[]
38,[https://arxiv.org/abs/1711.03480],[]
51,[https://doi.org/10.5534/wjmh.190077],[]


In [118]:
len(rr_subset)

67

In [119]:
#define function to count links
def count_links(entry):
    try:
        return len(entry)
    except TypeError:
        return 0

In [120]:
links_per_object = rr_subset.apply(
    lambda row: sum([count_links(entry) for entry in row]),
    axis=1
)

links_per_object

0      1
1      1
5      1
19     1
26     2
29     1
30     1
31     1
38     1
51     1
58     1
68     1
75     1
98     1
100    1
103    1
105    1
110    1
112    1
114    1
117    1
126    1
128    1
134    1
136    1
140    1
144    2
145    1
147    1
151    6
160    2
165    1
166    1
168    1
170    1
171    1
173    1
175    1
176    1
177    1
179    1
181    5
182    4
186    3
196    1
203    1
207    1
208    1
209    1
213    1
215    1
217    1
218    1
219    1
221    1
231    1
232    1
235    1
239    1
243    1
245    1
246    1
248    1
261    1
305    1
306    1
312    1
dtype: int64

In [121]:
print(f'mean {round(links_per_object.mean(), 3)} links per object')

mean 1.254 links per object


In [122]:
print(f'median {round(links_per_object.median(), 3)} links per object')

median 1.0 links per object


## 26. How many objects report each relation type? How many objects report multiple relation types, regardless of what those types are?
**Property:** Related resource relation type

In [123]:
relation_type = df.DataverseDatasetsCrosswalk.related_resource_relation_type
relation_type

In [124]:
#confirm missing for this repo
print(df.DataverseDatasetsCrosswalk.related_resource_relation_type)

None


## 27. For repositories that store the full citation in a designated field, how many objects have a populated citation? How many objects have a citation and a URL or other actionable link?
**Property:** Citation

In [125]:
citations = df.DataverseDatasetsCrosswalk.citation
citations

0      Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...
1      Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...
2      Charles, Grace, 2021, "Replication Data for: I...
3      Nag, Swapnika; Puri, Ankur; Brooks, Nathan; Ga...
4      Kim, Ungsoo, 2018, "Machine learn for glaucoma...
                             ...                        
305    Kvammen, Andreas; Wickstr&oslash;m, Kristoffer...
306    Senden, Mario; Emmerling, Thomas C.; van Hoof,...
312    Holland, Alisha, 2021, "Data for: Forbearance ...
313    The Norwegian Historical Data Centre, 2021, "M...
314    The Norwegian Historical Data Centre, 2021, "M...
Name: citationHtml, Length: 269, dtype: object

In [126]:
citations.describe()

count                                                   269
unique                                                  269
top       Axelrod, Simon; Gomez-Bombarelli, Rafael, 2021...
freq                                                      1
Name: citationHtml, dtype: object

In [127]:
print(f'Number of null descriptions: {sum(citations.isna())}')

Number of null descriptions: 0


In [128]:
print(f'All {len(citations)} objects have a citation')

All 269 objects have a citation
